# Librerias Necesarias

In [1]:
# Librerias externas
import json
import openai
import pandas as pd

from galois.chatgpt_galois import *
from galois.QueryTree import *
from galois.utils import *

# Funciones necesarias

In [2]:
def obtener_sql_plan(bd:str, verbose:bool, query:str) -> Node: 
    con = run_db(bd)
    con.execute("PRAGMA enable_profiling='query_tree';")
    con.execute("PRAGMA explain_output='ALL';")

    con.execute("EXPLAIN "+query.replace('"',"'"))
    s = con.fetchall()[0][1].split('\n')
    if verbose:
        print("\n".join(s))
        print('\n')
    root = parse_query_tree(s)
    
    return root

# Diccionario de queries para probar

In [8]:
join_query_trees={}
c0 = Node()
c0.text = ['SEQ_SCAN', 'nose']
c0.op = 'SEQ_SCAN'
c0.args = ['nose']

c1 = Node()
c1.text = ['PROJECTION', 'capital']
c1.op = 'PROJECTION'
c1.args = ['capital']
c1.filled_question = 'What is the capital of state of !!x!!?'

c1.l = c0

join_query_trees['SELECT t2.capital FROM state AS t2 JOIN city AS t1 ON t2.state_name = t1.state_name WHERE t1.city_name = "durham";'] = c1


# Ver query #1 en el archivo de planes
# join_query_trees['SELECT name FROM airports WHERE elevation BETWEEN -50 AND 50'] = obtener_sql_plan("data/spider_files/spider/database/flight_4/flight_4.sqlite", 
#                                                                                                      False, 
#                                                                                                      "SELECT name FROM airports WHERE elevation BETWEEN -50 AND 50")

# Traduccion de los operadores de SQL a NL

In [4]:
question_maps = json.load(open('data/question_maps.json','r'))
augmented_question_maps = augment_questions(question_maps)

# Configuración para chatgpt

In [5]:
# Instrucciones para chatgpt
inst_chatgpt = "You are a highly intelligent question answering bot. If I ask you a question that is rooted in truth, you will give you the answer. If I ask you a question that is nonsense, trickery, or has no clear answer, you will respond with 'Unknown'. You will answer concisely."

In [6]:
# Ejemplos para chatgpt
fewshot_chatgpt = [
 ['What is human life expectancy in the United States?', '78.'],
 ['Who was president of the United States in 1955?', 'Dwight D. Eisenhower.'],
 ['Which party was founded by Gramsci?', 'Comunista.'],
 ['What is the capital of France?', 'Paris.'],
 ['What is a continent starting with letter O?', 'Oceania.'],
 ['Where were the 1992 Olympics held?', 'Barcelona.'],
 ['How many squigs are in a bonk?', 'Unknown'],
 ['List me 5 fruits', 'apple, banana, peach, grape, grapefruit, strawberry'], 
 ['List me the first 5 prime numbers', '2, 3, 5, 7, 11'],
 ['List me 3 presidents of america', 'George Washington, Abraham Lincoln, Franklin D. Roosevelt']]

# Ejecutar galois

In [9]:
nombre= "experimento#"
contador = 1
for query, plan in join_query_trees.items():
    GPT_SPWJ(model_arch='gpt-3.5-turbo', 
            query=query,
            instr=inst_chatgpt,
            few_shots=fewshot_chatgpt,
            inst_funct=1,
            label=f'{nombre}{contador}',
            augmented_question_maps=augmented_question_maps,
            query_plan_dict=join_query_trees,
            verbose=True)

SELECT name FROM airports WHERE elevation BETWEEN -50 AND 50


Query in dict
Mode:  r
['SEQ_SCAN', 'airports']
Tree Nodes:  ['SEQ_SCAN_airports']
SEQ_SCAN_airports
OP:  SEQ_SCAN_airports
Q:  List some airports. Separate them by a comma. List as much as you can.
Unfiltered Answer:  ['JFK, LAX, ORD, ATL, LHR, CDG, DXB, HND, PEK, SIN']
Final Answer:  ['LHR', 'ORD', 'CDG', 'DXB', 'JFK', 'PEK', 'ATL', 'HND', 'SIN', 'LAX']


Mode:  r
['FILTER', '(elevation >= CAST(-50 AS BIGINT))', '(elevation <= CAST(50 AS BIGINT))']
Tree Nodes:  ['FILTER_(elevation >= CAST(-50 AS BIGINT))', 'FILTER_(elevation <= CAST(50 AS BIGINT))']
FILTER_(elevation >= CAST(-50 AS BIGINT))
OP:  FILTER_(elevation >= CAST(-50 AS BIGINT))
Q:  Is the elevation of !!x!! greater than or equal to -50? Answer with Yes or No only.
In Cache: 0/10
Added to cache: Is the elevation of LHR greater than or equal to -50? Answer with Yes or No only. 


Response:  No
Len of cache: 8
Added to cache: Is the elevation of ORD greater than or 